In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r'C:\Users\spiri\codingbootcamp\Final-project-C4Y\data.db')
cursor = conn.cursor()

In [ ]:
query1 = ('''
Select * 
from inv_campaign_eval
left join balances
on inv_campaign_eval.client_id = balances.client_id''')
dif_bal = pd.read_sql_query(query1, conn)
dif_bal = dif_bal.T.drop_duplicates().T

rate = {'CZK': 1, 'USD': 23, 'EUR': 25}
dif_bal.balance = dif_bal.apply(lambda row: row['balance'] * rate[row['currency']], axis=1)

dif_bal['balance_change'] = dif_bal.groupby('client_id')['balance'].diff()
dif_bal['balance_change'].fillna(0, inplace=True)

sum_dif = dif_bal.groupby('client_id')['balance_change'].apply(lambda arr: sum(x ** 2 for x in arr))
dif_bal = dif_bal.merge(sum_dif, on='client_id', how='left')
dif_bal

dif_bal.drop(columns=['date', 'balance', 'balance_change_x', 'currency'], inplace=True)
dif_bal.drop_duplicates(inplace=True)



,client_id,poutcome,currency,balance_change_y
0,249789938,success,CZK,128149.811845
26,1504633819,failure,CZK,119577.024704
52,1952195738,success,CZK,57924.635507
78,989726384,success,CZK,166963.270898
104,1971878545,failure,CZK,147742.223144
...,...,...,...,...
59644,1270855655,failure,CZK,111099.751243
59670,471724023,success,CZK,165275.194846
59696,620088171,failure,CZK,129786.760767
59722,1518319265,success,CZK,160942.081353
